In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import joblib
from tqdm.notebook import tqdm
from glob import glob
import pickle
from os import path
from unicodedata import normalize

In [2]:
path_name = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/1000_chunks_txt/*.txt'
path_test = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/test_chunks_txt/*.txt'
path_chunk = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/all_chunks_txt/1908_Rolland-Romain_Jean-Christophe_Tome-VII-Dans-la-maison.txt_chunk_43.txt'

In [3]:
path_verne = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/verne_txt/*.txt'
path_flaubert = "/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/flaubert_txt/*.txt"

In [4]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt).replace('\xa0', ' ')
    txt_res = txt_res.replace("\\", "").replace('\\xa0', '')
    return txt_res

In [1]:
def gpt_embeddings(chunk):
    client = OpenAI(
        api_key="[OPEN-AI-KEY]",
    )

    text = chunk.replace("\n", " ")
    embedding = client.embeddings.create(input = [text], model="text-embedding-ada-002").data[0].embedding
    
    return embedding


In [7]:
def get_embeddings(path_name):
    columns_df = ['doc']+[i for i in range(0,1536)]
    df_main = pd.DataFrame(columns=columns_df)

    for doc in tqdm(glob(path_name)):
        doc_name = path.splitext(path.basename(doc))[0]
        with open(doc, encoding="utf8") as file:
            text = file.readlines()
            text_cleaned = clean_text(str(text).lower())
            embedding = gpt_embeddings(text_cleaned)

            new_row_data = [doc_name] + embedding
            new_row_df = pd.DataFrame([new_row_data], columns=columns_df)

            df_main = pd.concat([df_main, new_row_df], ignore_index=True)
            
            df_temp = df_main.set_index('doc')
            df_temp.to_csv('GPT_flaubert_EMBEDDINGS_OUTPUT_TEMP.csv')
    
    df_main.set_index('doc', inplace=True)
    df_main.to_csv('GPT_flaubert_EMBEDDINGS_OUTPUT_MAIN.csv')
    
    return df_main

In [8]:
df_embeddings = get_embeddings(path_flaubert)

  0%|          | 0/153 [00:00<?, ?it/s]

In [ ]:
df_embeddings.to_csv('EMBEDDINGS_CHAPITRES_RANDOM.csv', index=True, header=True)